## Isolation Forest

Ансамбль деревьев решений.
Возвращает оценку аномалии каждой выборки с помощью алгоритма IsolationForest

In [995]:
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MaxAbsScaler,Normalizer,StandardScaler
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

In [996]:
RANDOM_STATE = 42
sns.set(style="darkgrid")
pd.set_option('display.max_rows', 1000)
data = pd.read_csv("all_eda_obr20_04-2.csv")
data["TimeU"]
#list(data.columns)

0        1381827300
1        1381827310
2        1381827320
3        1381827330
4        1381827340
            ...    
21561    1296585150
21562    1296585160
21563    1296585170
21564    1296585180
21565    1296585190
Name: TimeU, Length: 21566, dtype: int64

In [1355]:
# Может запилить GridSearch для поиска оптим.гиперпараметров?

outliers_fraction = 0.05   #регулируем    0.07 
data = pd.read_csv("all_eda_obr20_04-2.csv")

data1 =  data[data['RIG_STATE']==7]
df = data1[['DEPT_bin', 'SPPA_bin', 'BVEL_bin', 'HKLD_bin', 'BPOS_bin',"TimeU"]] #Павел
df6 = data1[['DEPT_bin', 'SPPA_bin', 'BVEL_bin', 'HKLD_bin', 
            'BPOS_bin', "RPM_bin", "STOR_bin"]] #Костя 
 
df2 = data1[['BPOS', 'FBPOS_1' ,'FBPOS_3' ,'DHKLD_6' ,'DSPPA_3','DSPPA_6' ,
             'DEPT_bin','SPPA_bin','SPPA_APRS_bin',
             'BVEL_bin','HKLD_bin','ANOMAL','BPOS_bin',"TimeU"]] #Татьяна

#pca = PCA(n_components=4)
#df2 = pca.fit_transform(df2)
model =  IsolationForest(n_estimators=70, 
                         max_samples = 70,#регулируем
                        contamination = outliers_fraction)
model.fit(df2)
labels = model.predict(df2)
labels
#clf = IsolationForest(random_state=0).fit(df2)
#labels = clf.predict(df2)

array([ 1,  1,  1, ...,  1, -1, -1])

In [1356]:
df9 = pd.DataFrame()
df9['StuckPipe'] = data1['StuckPipe']
df9['predict'] = labels
df9.value_counts()

StuckPipe  predict
0.0         1         9964
1.0        -1          274
0.0        -1          260
1.0         1          171
dtype: int64

In [1357]:
df9['predict'].unique()

array([ 1, -1])

In [1358]:
df9['predict']=df9['predict'].apply(lambda n: 0 if n==1 else 1)
df9['predict'].value_counts()

0    10135
1      534
Name: predict, dtype: int64

In [1359]:
df9.value_counts()

StuckPipe  predict
0.0        0          9964
1.0        1           274
0.0        1           260
1.0        0           171
dtype: int64

In [1360]:
metrics.f1_score(data1['StuckPipe'], df9['predict'])

0.5597548518896834

In [1361]:
print(classification_report(data1['StuckPipe'], df9['predict']))

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98     10224
         1.0       0.51      0.62      0.56       445

    accuracy                           0.96     10669
   macro avg       0.75      0.80      0.77     10669
weighted avg       0.96      0.96      0.96     10669



## Проверим модель на 1 скважине: (получается лучше результат на отдельных скважинах)

In [1004]:
outliers_fraction = 0.07 #регулируем    0.07 
data = pd.read_csv("all_eda_obr20_04-2.csv")

data1 =  data[data['RIG_STATE']==7]
data1 =  data1[data1['hole']=="241"]   # 241
df2 = data1[['DEPT_bin', 'SPPA_bin', 'BVEL_bin', 'HKLD_bin', 'BPOS_bin']]
df77 = data1[[ 'DEPT_bin','SPPA_bin',  'HKLD_bin',
            'BPOS_bin', "RPM_bin",'DEPT_bin',
            ]] #"STOR_bin",'BPOS_bin', "RPM_bin"
df2 = data1[['BPOS', 'FBPOS_1' ,'FBPOS_3' ,'DHKLD_6' ,'DSPPA_3','DSPPA_6' ,
             'DEPT_bin','SPPA_bin','SPPA_APRS_bin' ,
             'BVEL_bin','HKLD_bin','ANOMAL','BPOS_bin']]
#pca = PCA(n_components=2)
#df2 = pca.fit_transform(df2)
model =  IsolationForest(n_estimators=8, 
                         max_samples = 50,#регулируем
                        contamination = outliers_fraction)
model.fit(df2)
labels = model.predict(df2)
#labels

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [1005]:
df9 = pd.DataFrame()
df9['StuckPipe'] = data1['StuckPipe']
df9['predict'] = labels
df9.value_counts()

StuckPipe  predict
0.0         1         488
1.0        -1          19
0.0        -1          17
1.0         1           5
dtype: int64

In [1006]:
df9['predict']=df9['predict'].apply(lambda n: 0 if n==1 else 1)
df9['predict'].value_counts()

0    493
1     36
Name: predict, dtype: int64

In [1007]:
metrics.f1_score(data1['StuckPipe'], df9['predict'])

0.6333333333333333

In [1008]:
print(classification_report(data1['StuckPipe'], df9['predict']))

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98       505
         1.0       0.53      0.79      0.63        24

    accuracy                           0.96       529
   macro avg       0.76      0.88      0.81       529
weighted avg       0.97      0.96      0.96       529



### Результат неплохой:  Recall 79%!!!

In [1009]:
data.hole.unique()

array(['204', '210ST', '210ST2', '216', '217ST1', '217ST2', '220',
       '220ST3', '220ST4', '234ST2', '241', '243', '256ST2', '256ST3',
       '278', '279'], dtype=object)

In [1016]:
outliers_fraction = 0.06 #регулируем    0.07 , 0.09
data = pd.read_csv("all_eda_obr20_04-2.csv")

data1 =  data[data['RIG_STATE']==7]
data1 =  data1[data1['hole']=="241"]   # 241             
df2 = data1[['DEPT_bin', 'SPPA_bin', 'BVEL_bin', 'HKLD_bin', 'BPOS_bin',"TimeU"]]

df1 = data1[['BPOS', 'FBPOS_1' ,'FBPOS_3' ,'DHKLD_6' ,'DSPPA_3','DSPPA_6' ,
             'DEPT_bin','SPPA_bin','SPPA_APRS_bin',
             'BVEL_bin','HKLD_bin','ANOMAL','BPOS_bin',"TimeU"]] #Tatjana


#pca = PCA(n_components=2)
#df2 = pca.fit_transform(df2)
model =  IsolationForest(n_estimators=50,  # регулируем  15/5
                          max_samples = 50, # регулируем  90
                        contamination = outliers_fraction)
model.fit(df2)
labels = model.predict(df2)
#labels

In [1017]:
df9 = pd.DataFrame()
df9['StuckPipe'] = data1['StuckPipe']
df9['predict'] = labels
df9.value_counts()

StuckPipe  predict
0.0         1         494
1.0        -1          18
0.0        -1          11
1.0         1           6
dtype: int64

In [1018]:
df9['predict']=df9['predict'].apply(lambda n: 0 if n==1 else 1)
df9['predict'].value_counts()

0    500
1     29
Name: predict, dtype: int64

In [1019]:
df9.value_counts()

StuckPipe  predict
0.0        0          494
1.0        1           18
0.0        1           11
1.0        0            6
dtype: int64

In [1020]:
metrics.f1_score(data1['StuckPipe'], df9['predict'])

0.679245283018868

### Результат неплохой:  Recall 75%!!!

In [1021]:
from sklearn.metrics import classification_report
#classification_report(df9['predict'],data1['StuckPipe'])
print(classification_report(data1['StuckPipe'], df9['predict']))

              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98       505
         1.0       0.62      0.75      0.68        24

    accuracy                           0.97       529
   macro avg       0.80      0.86      0.83       529
weighted avg       0.97      0.97      0.97       529



## One class SVM¶

In [1399]:

data = pd.read_csv("all_eda_obr20_04-2.csv")

data1 =  data[data['RIG_STATE']==7]
#data1 =  data1[data1['hole']=="241"]   # 241             
df2 = data1[['DEPT_bin', 'SPPA_bin',"RPM_bin","HKLD_bin"]]  #интересный результат Recall!!!! 71%

# train one class SVM 
clf = OneClassSVM(gamma='scale').fit(df2)
model.fit(df2)
labels = clf.predict(df2)
clf.score_samples(df2)


array([3695.84342443, 3695.84342443, 3695.84342443, ...,  934.185006  ,
        327.36507391,  934.185006  ])

In [1400]:
labels

array([ 1,  1,  1, ..., -1, -1, -1])

In [1401]:
df9 = pd.DataFrame()
df9['StuckPipe'] = data1['StuckPipe']
df9['predict'] = labels
df9.value_counts()

StuckPipe  predict
0.0         1         8913
           -1         1311
1.0        -1          331
            1          114
dtype: int64

In [1402]:
# ПРиведем значения labels под наш Таргет:
df9['predict']=df9['predict'].apply(lambda n: 0 if n==1 else 1)
df9['predict'].value_counts()

0    9027
1    1642
Name: predict, dtype: int64

In [1403]:
df9.value_counts()

StuckPipe  predict
0.0        0          8913
           1          1311
1.0        1           331
           0           114
dtype: int64

In [1404]:
from sklearn.metrics import classification_report
#classification_report(df9['predict'],data1['StuckPipe'])
print(confusion_matrix(data1['StuckPipe'], df9['predict']))
print(classification_report(data1['StuckPipe'], df9['predict']))

[[8913 1311]
 [ 114  331]]
              precision    recall  f1-score   support

         0.0       0.99      0.87      0.93     10224
         1.0       0.20      0.74      0.32       445

    accuracy                           0.87     10669
   macro avg       0.59      0.81      0.62     10669
weighted avg       0.95      0.87      0.90     10669



### Результат неплохой:  Recall 71%!!!

In [1398]:
metrics.f1_score(data1['StuckPipe'], df9['predict'])

0.29428228708516596